# Dataset Utilities

In the following notebook we will demonstrate several utilities provided by RankEval in working and manipulating a Dataset in the SVMLight format. In particular, how to:
 - Easily Load standard LtR datasets as well as several pre-trained models
 - Fork a dataset by selecting only a subset of the features
 - Fork a dataset by selecting only some queries
 - Dump a dataset on file in the SVMLight format
 - Split a dataset in train, validation (eventually) and test sets
 - Manually accessing low level dataset information
 - Iterate over each query of a dataset

#### Essential imports

In [3]:
from __future__ import print_function

%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd

# Useful to reload the module without having to restart the notebook kernel
from rankeval.dataset.datasets_fetcher import load_dataset
from rankeval.dataset import Dataset
from rankeval.model import RTEnsemble

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading datasets and models

Standard LtR datasets can be easily loaded by calling the load_dataset utility. This tool allows to load datasets and several pre-trained models from a central repository, in such a way to simplify the setting of the workspace.

In [4]:
# Dataset container
dataset_container = load_dataset(dataset_name='msn10k', 
                                fold='1', 
                                download_if_missing=True, 
                                force_download=False,
                                with_models=True)

Loading files. This may take a few minutes.
done loading dataset!


## Datasets

In [5]:
# Remapping Datasets Names
msn_train = dataset_container.train_dataset
msn_validation = dataset_container.validation_dataset
msn_test = dataset_container.test_dataset

## Choose and load models

In [6]:
# View available models
for item, file_name in enumerate(dataset_container.model_filenames):
    print (item, file_name)

0 /Users/salvatore/rankeval_data/msn10k/models/Fold1/quickrank/msn1.quickrank.LAMBDAMART.20000.32.T15000.xml
1 /Users/salvatore/rankeval_data/msn10k/models/Fold1/quickrank/msn1.quickrank.LAMBDAMART.20000.32.T5000.xml
2 /Users/salvatore/rankeval_data/msn10k/models/Fold1/quickrank/msn1.quickrank.LAMBDAMART.20000.32.T20000.xml
3 /Users/salvatore/rankeval_data/msn10k/models/Fold1/quickrank/msn1.quickrank.LAMBDAMART.20000.32.T10000.xml
4 /Users/salvatore/rankeval_data/msn10k/models/Fold1/quickrank/msn1.quickrank.LAMBDAMART.20000.32.T1000.xml
5 /Users/salvatore/rankeval_data/msn10k/models/Fold1/xgboost/XGBOOST.msn10k.fold-1.pairwise.d-5.lr-10.trees-1000.model
6 /Users/salvatore/rankeval_data/msn10k/models/Fold1/lightgbm/LGBM.msn10k.fold-1.lambdarank.leaves-32.lr-5.trees-1000.model
7 /Users/salvatore/rankeval_data/msn10k/models/Fold1/catboost/msn1.catboost.LAMBDAMART.1000.5.T1000.json
8 /Users/salvatore/rankeval_data/msn10k/models/Fold1/catboost/msn1.catboost.LAMBDAMART.1000.5.T1000.model


In [7]:
# Model files
msn_qr_lmart_1Ktrees_file = dataset_container.model_filenames[4]
# Loading model into RankEval
msn_qr_lmart_1Ktrees = RTEnsemble(msn_qr_lmart_1Ktrees_file, name="QR_lmart_1K", format="QuickRank")

# Fork a dataset by selecting only a subset of the features

Starting from a dataset, it is possible to create a new dataset instance with only a subset of the features appearing in the original dataset. 
This feature is particularly useful when the task is to analyze the feature importance, trying to reduce as much as possible the features needed by a LtR model without affecting the quality of the learned model. 
An example of such analysis is reported in the notebook: [Feature Analysis](Feature%20Analysis.ipynb).

In this notebook the features selected will be the 20% of all the features, randomly choosen:

In [8]:
feature_permutation = np.random.permutation(msn_train.n_features)
selected_features = feature_permutation[:int(msn_train.n_features*0.2)]

msn_train_subset_features = msn_train.subset_features(selected_features)

In [9]:
d = {'# Queries': [msn_train.n_queries, msn_train_subset_features.n_queries], 
     '# Instances': [msn_train.n_instances, msn_train_subset_features.n_instances], 
     '# Features': [msn_train.n_features, msn_train_subset_features.n_features],}
df = pd.DataFrame(data=d, index=["Full", "Sampled by Feature"])
df

,# Queries,# Instances,# Features
Full,6000,723412,136
Sampled by Feature,6000,723412,27


Recall the models have to be used only in consistent conditions, i.e., if you train a model on this sampled dataset also the test dataset (and eventually the validation) has to be changed accordingly.

In [10]:
msn_test_subset_features = msn_test.subset_features(selected_features)

# Fork a dataset by selecting only a subset of the queries

Starting from a dataset, it is possible to create a new dataset instance by selecting only some of the queries of the original dataset. The query to select are identified by qid (query id).

In this notebook we select only 20% of all the queries, randomly choosen:

In [11]:
qid_permutation = np.random.permutation(msn_train.query_ids)
selected_qid = qid_permutation[:int(msn_train.query_ids.size*0.2)]

msn_train_subset_queries = msn_train.subset(query_ids=selected_qid, name="MSN Train Fold1 - 20% of the queries")

In [12]:
d = {'# Queries': [msn_train.n_queries, msn_train_subset_queries.n_queries], 
     '# Instances': [msn_train.n_instances, msn_train_subset_queries.n_instances], 
     '# Features': [msn_train.n_features, msn_train_subset_queries.n_features],}
df = pd.DataFrame(data=d, index=["Full", "Sampled by Query"])
df

,# Queries,# Instances,# Features
Full,6000,723412,136
Sampled by Query,1200,145114,136


# Dump a model on file

If you modify a dataset (by selecting only a subset of the queries or of the features) you can save the modified version on file in the standard SVMLight format. This operation is provided by RankEval to simplify the activity of manipulating and working on LtR datasets.

In [13]:
destination_file = "msn_fold1_27_random_features.txt"
msn_train_subset_features.dump(destination_file, format="svmlight")

Read the top 10 lines of the file where the dataset has been written into (and delete it):

In [14]:
!head -10 $destination_file
!rm $destination_file

2 qid:1 1:0 2:1 3:0.011976 4:0 5:5.9924598 6:0 7:16.76696 8:2.2971971 9:0 10:0 11:-21.339609 12:0 13:6.1565952 14:13 15:167 16:1 17:0.011976 18:12.941469 19:0.019231001 20:0 21:6.9265509 22:3.078917 23:0 24:0.0064099999 25:0 26:2 27:0 
2 qid:1 1:0 2:0 3:0.021635 4:0.88888901 5:0 6:0.88888901 7:21.161667 8:12.634581 9:0.60000002 10:0 11:-24.805464 12:7.9857841 13:33.861275 14:1 15:416 16:0 17:0.026442001 18:20.885118 19:0.068966001 20:0 21:6.9265509 22:30.789171 23:0 24:0.024630999 25:0 26:10.333333 27:0 
0 qid:1 1:0 2:0 3:0.0064099999 4:6.8888888 5:0 6:9.5555563 7:18.279205 8:2.6996551 9:0.66666698 10:0 11:-24.805464 12:6.3144059 13:21.548082 14:14 15:156 16:0 17:0.051282 18:18.140879 19:0.09589 20:0 21:6.9265509 22:18.473503 23:0 24:0.047945 25:0 26:5.333333 27:0.024691001 
2 qid:1 1:0 2:0 3:0.010033 4:0.222222 5:0 6:0.222222 7:17.53163 8:4.5943928 9:0.75 10:0 11:-24.805464 12:7.9857841 13:9.2348928 14:1 15:299 16:0 17:0.013378 18:15.572998 19:0.024390001 20:0 21:6.9265509 22:6.157834

# Split a dataset in train, validation (eventually) and test sets

It is common the need to split a LtR dataset in train, validation and test sets as to train a LtR model and test its effectiveness on a different split.
Rankeval provides an utility to split a dataset into partitions. It shuffle the query ids before partitioning.

In this notebook for simplicity we split the training set of the MSN-Fold1 dataset according to the 60%/20%/20% ratios.

In [15]:
train, vali, test = msn_train.split(train_size=0.6, vali_size=0.2)

In [16]:
d = {'# Queries': [train.n_queries, vali.n_queries, test.n_queries], 
     '# Instances': [train.n_instances, vali.n_instances, test.n_instances], 
     '# Features': [train.n_features, vali.n_features, test.n_features],}
df = pd.DataFrame(data=d, index=["Train", "Vali", "Test"])
df

,# Queries,# Instances,# Features
Train,3600,432458,136
Vali,1200,144283,136
Test,1200,146671,136


If `vali_size` is set to 0, the method will skip the creation of a validation set.

# Accessing low-level information

The dataset class provides an interface storing dataset information and providing utilities for easily accessing it. In some situation however it is needed to access the low level information there stored. In this notebook, we describe how this information is stored, and how to iterate over the queries of a dataset accessing features, labels and query ids in order.

We start by describing the base component of a dataset instance.

This are the main charateristics of the dataset we are adopting:

In [17]:
d = {'# Queries': [msn_train.n_queries], 
     '# Instances': [msn_train.n_instances], 
     '# Features': [msn_train.n_features],}
df = pd.DataFrame(data=d, index=["Dataset"])
df

,# Queries,# Instances,# Features
Dataset,6000,723412,136


Each dataset is described in primis by the feature matrix (numpy 2d array) of its feature. The rows of this matrix are the instances, whether the columns are the features. The shape of the matrix is thus (n_instances, n_features):

In [18]:
msn_train.X.shape

(723412, 136)

Each cell of this matrix identify the value of a single feature/document pair. For example, the value of the 10-th feature of the 5-th instance is (remember indices starts from 0):

In [19]:
msn_train.X[4, 9]

1.0

The ground truth labels are stored in a separate vector (numpy 1d array), with a single value for each dataset instance. The shape of the vector is thus (n_instances):

In [20]:
msn_train.y.shape

(723412,)

Each cell of this vector identify the ground truth value of a single instance for the specific query. For example, the ground truth value of the 5-th instance is (recall ground truth labels usually range in [0-4], with 0 meaning completely irrelevant and 4 perfectly relevant):

In [21]:
msn_train.y[4]

1.0

The query ids of the dataset are stored in a separate vector. This information is not strictly needed, but when you manipulate a dataset sometime it is important to preserve this information (e.g., for comparison).
This information is stored in a vector (numpy 1d array) with a single value for each dataset query. The shape of the vector is thus (n_queries):

In [22]:
msn_train.query_ids.shape

(6000,)

The query id of the first query is:

In [23]:
msn_train.query_ids[0]

1

The offsets of each query are on the other hand stored in another vector (numpy 1d array). These offsets allows to associate each dataset instance to the correct query. Recall indeed that both features and ground truth labels are stored contiguosly between several queries, thus it is needed a mechanism allowing to discriminate and reconstruct the information regarding each query.

This array stores the starting offset of each query (starting from the first instance identified by the first row). Its shape is thus (n_queries+1), with the latter element that is not strictly needed but is put there for simplicity (its value is always the number of instances in the dataset).

In [24]:
msn_train.query_offsets.shape

(6001,)

Given this information, the i-th query has indices ranging from query_offsets[i] up to query_offsets[i+1], with the latter element excluded. For example, the first query start at index 0 and contains 86 documents up to index 86 (excluded):

In [25]:
msn_train.query_offsets[0:2]

array([ 0, 86])

The number of documents in each query can thus be obtained automatically from the `query_offset` variable. However, RankEval provide a simple utility for accessing this information, that is the `get_query_sizes` method:

In [26]:
msn_train.get_query_sizes()

array([ 86, 106,  92, ...,  79, 180,  40])

This method returns a vector (numpy 1d array) with a single element for each query, identifying the number of documents belonging to this query. As noticed above, the first query has 86 documents, the second has 106 documents, and so on and so forth. The shape of this vector is thus (n_queries):

In [27]:
msn_train.get_query_sizes().shape

(6000,)

In some situation we need to have the list of query ids, one for each instance (in place of the compact version adopted by RankEval). For example, this feature is needed by LightGBM when you need to create a dataset. RankEval provides such information with the `get_qids_dataset` utility, returning a vector (numpy 1d array) with shape (n_instances):

In [28]:
msn_train.get_qids_dataset().shape

(723412,)

The first document has qid:

In [29]:
msn_train.get_qids_dataset()[0]

1

# Iterate over each query of a dataset

Sometime it is needed to iterate over the low level information stored in the dataset instance. RankEval allows you to do that with the `query_iterator` utility, that provides you with an iterator over the offsets of the query_ids in the dataset. In particular, each element of the iterator is a tuple (qid, start_offset, end_offset), highlighting thus the qid of the query and the row index of the instances belonging it. For example, the first element of the iterator is related to the query with `qid=1` and offset ranging from 0 to 86 (excluded):

In [30]:
next(msn_train.query_iterator())

(1, 0, 86)